In [1]:
import sys
sys.path.append("../src")

In [2]:
import captum.attr as a
from torch.utils.data import DataLoader

from dataset import OurDataset
from explain import STS_ExplainWrapper, ExplanationExecuter

KeyboardInterrupt: 

In [3]:
dataset = OurDataset(csv_dirpath="./data", split="test")

#smaller ds for testing purposes
dataset.fact_check_post_mapping = dataset.fact_check_post_mapping[:20] 
loader = DataLoader(dataset, batch_size=1)

In [1]:
# remove_interpretable_embedding_layer

In [4]:
model = STS_ExplainWrapper.setup_t5_transformer("../models/GTR-T5-FT",interpretable_embeddings=False)
method = a.LayerGradientXActivation(model, layer=model.get_embedding_layer())
explain = ExplanationExecuter(method, compute_baseline=False, visualize_explanation=False)

In [4]:
model = STS_ExplainWrapper.setup_t5_transformer("../models/GTR-T5-FT",interpretable_embeddings=True)


c:\Users\Dell\anaconda3\envs\projectXAI\lib\site-packages\captum\attr\_models\base.py:191: UserWarning: In order to make embedding layers more interpretable they will be replaced with an interpretable embedding layer which wraps the original embedding layer and takes word embedding vectors as inputs of the forward function. This allows us to generate baselines for word embeddings and compute attributions for each embedding dimension. The original embedding layer must be set back by calling `remove_interpretable_embedding_layer` function after model interpretation is finished. 
  warnings.warn(


In [5]:
import torch.nn as nn
from captum.attr._utils.lrp_rules import EpsilonRule,Alpha1_Beta0_Rule
from transformers.models.t5.modeling_t5 import T5LayerNorm  
from sentence_transformers.models import Pooling 


def apply_propagation_rule(module, rule):
    if isinstance(module, (nn.Embedding, nn.modules.sparse.Embedding,nn.Linear,nn.LayerNorm,nn.Dropout,T5LayerNorm,Pooling,nn.ReLU)):
        module.rule = rule

    for child in module.children():
        apply_propagation_rule(child, rule)

epsilon_rule = Alpha1_Beta0_Rule()
apply_propagation_rule(model, epsilon_rule)

In [6]:
def hook_fn(module, input, output):
    if not hasattr(module, "call_count"):
        module.call_count = 0
    module.call_count += 1
    print(f"Module: {module}, Call Index: {module.call_count}")

for module in model.modules():
    if isinstance(module, nn.Dropout):
        module.register_forward_hook(hook_fn)


In [7]:
method = a.LRP(model)
explain = ExplanationExecuter(method, compute_baseline=False, visualize_explanation=False)

In [41]:
# list(
    
model.model.transformer.hf_transformer.encoder.embed_tokens.embedding

# .activations.values()

# )[0][0].size()

{}

In [ ]:
# layer.embedding need to transfer layer to layer.embedding for model to process layer methods

In [8]:
explanation = explain.explain(loader, return_post_explanation_only=True)

  0%|          | 0/20 [00:00<?, ?it/s]

Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call Index: 1
Module: Dropout(p=0.1, inplace=False), Call In

c:\Users\Dell\anaconda3\envs\projectXAI\lib\site-packages\captum\_utils\gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
  0%|          | 0/20 [00:02<?, ?it/s]


Module: Dropout(p=0.1, inplace=False), Call Index: 5


RuntimeError: Module Dropout(p=0.1, inplace=False) is being used more than once in the network, which is not supported by LRP. Please ensure that module is being used only once in the network.

[array([[7.70932257e-01, 7.94427872e-01, 9.09391761e-01, 7.76501536e-01,
         1.00000000e+00, 7.88260698e-01, 7.20296204e-01, 1.00000000e+00,
         5.18496573e-01, 4.78099018e-01, 5.98549843e-01, 2.93115765e-01,
         5.84849417e-02, 0.00000000e+00, 6.10947132e-01, 8.39827716e-01,
         8.74606311e-01, 6.26933098e-01, 7.84492586e-05, 7.23885596e-01,
         8.38333845e-01, 8.85437999e-04, 8.63965452e-01, 7.99660861e-01,
         5.62956572e-01, 8.73776674e-01, 9.68794703e-01, 6.28110766e-01,
         1.00000000e+00, 5.36271334e-01, 8.90051499e-02, 4.59269702e-01,
         5.28498411e-01, 5.50099611e-01, 5.75449586e-01, 6.12481296e-01,
         8.78108561e-01, 0.00000000e+00, 6.49187326e-01, 8.90142739e-01,
         6.97020054e-01, 5.88894784e-01, 4.41289902e-01, 3.85944396e-01,
         4.70207751e-01, 2.70010740e-01, 6.10793948e-01, 8.33425641e-01,
         5.14223576e-01, 5.08353531e-01, 3.50141168e-01, 4.67350692e-01,
         4.80060101e-01, 7.41279483e-01, 8.98036957

In [ ]:
import sys
sys.path.append("../src")

In [16]:
import captum.attr as a
from torch.utils.data import DataLoader

from dataset import OurDataset
from explain import STS_ExplainWrapper, ExplanationExecuter,compare_multiple_explanation_methods

In [ ]:
dataset = OurDataset(csv_dirpath="./data", split="test")

#smaller ds for testing purposes
dataset.fact_check_post_mapping = dataset.fact_check_post_mapping[:20] 
loader = DataLoader(dataset, batch_size=1)

In [14]:
model = STS_ExplainWrapper.setup_t5_transformer("../models/GTR-T5-FT")
method = a.LayerGradientXActivation(model, layer=model.get_embedding_layer())
# explain = ExplanationExecuter(method, compute_baseline=False, visualize_explanation=False)

In [15]:
claim, post = dataset[0]

In [22]:
explain_wrappers=[]
explain_wrappers.append(ExplanationExecuter(method,visualize_explanation=False,apply_normalization=False))
compare_multiple_explanation_methods(explain_wrappers, post, claim, additional_attribution_kwargs= {}, method_names=[method],visualize=False)

--- Explaining mutual similarity between a [POST] and a [CLAIM] ---
	[POST]: You still drinking Dasani? Dasani is considered the worst bottled water in the market because it contains potassium chloride. It's the same chemical that's given to death row inmates before they die. Extended exposure to this may lead to weaker bones & cardiac arrest. @The FooCommunity PRIFIED WATER AFFORAPURE, FRESH TAST PURIFIED WATER WEDSVE 16.9 FL OZ CLOSP 500 ml plant bottle PURIFIED WATER ENHANCED WITH MINERALS FOR & PURE, FREN LOG PT) 500 ml. ASANISANI DASANI DASANASANT plonbo O plantball 0 CARES CARTIE D plant bottle ATER 16.9 FL OZ (106 PT) 500 ml NEFES S 18464 PURIFIED WATER ENHANCED WITH MINERALS FOR 0 CALORES PERATR PURIFIED WATER FINALS FOR A PURE, FRESH TASSE 16.9 FLOZ (LOG PI) 500 ml. plant bottles KOPLOZ (L06 PT) 500 ml
	[CLAIM]: Additives In Dasani Bottled Water Are Dangerous To Drink
------- Explaining relevance of [POST] -------
------- Explaining relevance of [CLAIM] -------


[{'explanation': {<captum.attr._core.layer.layer_gradient_x_activation.LayerGradientXActivation at 0x22cc4ba4b20>: (tensor([[ 4.3256e-04,  4.8499e-04,  7.4156e-04,  4.4498e-04,  1.6017e-03,
              4.7122e-04,  3.1958e-04,  1.6471e-03, -1.3056e-04, -2.2064e-04,
              4.7986e-05, -6.3305e-04, -1.1559e-03, -1.7696e-03,  7.5638e-05,
              5.8630e-04,  6.6392e-04,  1.1130e-04, -1.2860e-03,  3.2759e-04,
              5.8297e-04, -1.2842e-03,  6.4017e-04,  4.9666e-04, -3.1402e-05,
              6.6207e-04,  8.7416e-04,  1.1392e-04,  1.4869e-03, -9.0917e-05,
             -1.0879e-03, -2.6263e-04, -1.0825e-04, -6.0077e-05, -3.5381e-06,
              7.9060e-05,  6.7173e-04, -1.7524e-03,  1.6094e-04,  6.9859e-04,
              2.6765e-04,  2.6450e-05, -3.0272e-04, -4.2612e-04, -2.3824e-04,
             -6.8455e-04,  7.5296e-05,  5.7201e-04, -1.4009e-04, -1.5318e-04,
             -5.0594e-04, -2.4461e-04, -2.1627e-04,  3.6639e-04,  7.1621e-04,
             -1.3854e-03,  9.4

In [6]:
print("Num batches:", len(explanation))
print("Explanation shape of batch 0:", explanation[0].shape)

Num batches: 5
Explanation shape of batch 0: (4, 265)


In [12]:
explanation[0]

array([[7.70932257e-01, 7.94427872e-01, 9.09391761e-01, 7.76501536e-01,
        1.00000000e+00, 7.88260698e-01, 7.20296204e-01, 1.00000000e+00,
        5.18496573e-01, 4.78099018e-01, 5.98549843e-01, 2.93115765e-01,
        5.84849417e-02, 0.00000000e+00, 6.10947132e-01, 8.39827716e-01,
        8.74606311e-01, 6.26933098e-01, 7.84492586e-05, 7.23885596e-01,
        8.38333845e-01, 8.85437999e-04, 8.63965452e-01, 7.99660861e-01,
        5.62956572e-01, 8.73776674e-01, 9.68794703e-01, 6.28110766e-01,
        1.00000000e+00, 5.36271334e-01, 8.90051499e-02, 4.59269702e-01,
        5.28498411e-01, 5.50099611e-01, 5.75449586e-01, 6.12481296e-01,
        8.78108561e-01, 0.00000000e+00, 6.49187326e-01, 8.90142739e-01,
        6.97020054e-01, 5.88894784e-01, 4.41289902e-01, 3.85944396e-01,
        4.70207751e-01, 2.70010740e-01, 6.10793948e-01, 8.33425641e-01,
        5.14223576e-01, 5.08353531e-01, 3.50141168e-01, 4.67350692e-01,
        4.80060101e-01, 7.41279483e-01, 8.98036957e-01, 0.000000